## ASIDE SECTION 1 - processing to verify all the train2017 files are uploaded
### should be 118287 images

### have already uploaded the files from laptop to gdrive using my own script

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls

gdrive	sample_data


In [ ]:
!ls './gdrive/My Drive/ThesisStoryGen/Data/coco_train2017_wget_1/'

COCO_train2017_118k  Rishabh_DO_NOT_TOUCH_PLS_1  train2017.zip


In [ ]:
!ls './gdrive/My Drive/ThesisStoryGen/Data/coco_train2017_wget_1/Rishabh_DO_NOT_TOUCH_PLS_1' | wc -l

ls: cannot open directory './gdrive/My Drive/ThesisStoryGen/Data/coco_train2017_wget_1/Rishabh_DO_NOT_TOUCH_PLS_1': Input/output error
0


In [ ]:
import os
import pickle
import sys

In [ ]:
IMGSDIR_COCO_TRAIN2017 = r'./gdrive/My Drive/ThesisStoryGen/Data/coco_train2017_wget_1/Rishabh_DO_NOT_TOUCH_PLS_1'
WORKDIR_COCO_TRAIN2017 = '/content/gdrive/My Drive/ThesisStoryGen/Data/ImgCapTraining/CocoTrain2017_1/'

In [ ]:
gdrive_coco_train2017_118287_listdir = os.listdir(IMGSDIR_COCO_TRAIN2017)

In [ ]:
len(gdrive_coco_train2017_118287_listdir)

118287

In [ ]:
## pickle the data
with open(WORKDIR_COCO_TRAIN2017+'gdrive_coco_train2017_118287_listdir.pkl', 'wb') as handle:
  pickle.dump(gdrive_coco_train2017_118287_listdir, handle, protocol=pickle.HIGHEST_PROTOCOL)
print(f"Pickling done")

Pickling done


In [ ]:
print(f"Size of pickled data = {sys.getsizeof(gdrive_coco_train2017_118287_listdir)} bytes")

Size of pickled data = 1043560 bytes


In [ ]:
del gdrive_coco_train2017_118287_listdir

In [ ]:
## reload from picked file - output of listdir on google drive location
if True:
  with open(WORKDIR_COCO_TRAIN2017+'gdrive_coco_train2017_118287_listdir.pkl', 'rb') as handle:
    gdrive_coco_train2017_118287_listdir = pickle.load(handle)

len(gdrive_coco_train2017_118287_listdir)

118287

In [ ]:
## reload from picked file - output of listdir on laptop folder
if True:
  with open(WORKDIR_COCO_TRAIN2017+'laptop_coco_train2017_listdir.pkl', 'rb') as handle:
    laptop_coco_train2017_listdir = pickle.load(handle)

len(laptop_coco_train2017_listdir)

118287

In [ ]:
upload_pending = list( set(gdrive_coco_train2017_118287_listdir) - set(laptop_coco_train2017_listdir) )

In [ ]:
len(upload_pending)  ## if 0, then the images in the folders on the gdrive and laptop match

0

In [ ]:
## double check that all values in the listdir of gdrive are unique

In [ ]:
import pandas as pd

In [ ]:
dfup = pd.DataFrame(gdrive_coco_train2017_118287_listdir, columns=['fname'])
dfup.head()

,fname
0,000000407952.jpg
1,000000031471.jpg
2,000000208524.jpg
3,000000480872.jpg
4,000000257301.jpg


In [ ]:
dfup['fname'].nunique()  ## should be 118287

118287

In [ ]:
len(dfup)   ## also should be 118287

118287

## ASIDE SECTION 2 - BREAK COCO TRAIN2017 118k FILES INTO SMALLER PARTS IN FOLDERS
### total images in COCO_TRAIN2017 = 118287 jpgs

### using the start and end indexes, using the os.listdir output on the folder
###     create a new folder and copy that set of images os.listdir(xxx)[sidx:eidx]

In [1]:
import pickle
import os
import sys
import shutil
import datetime
import time

In [2]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/gdrive')

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/gdrive


In [3]:
COCO_TRAIN2017_IMGSDIR = r'./gdrive/My Drive/ThesisStoryGen/Data/coco_train2017_wget_1/Rishabh_DO_NOT_TOUCH_PLS_1/'
WORKDIR_COCO_TRAIN2017 = r'./gdrive/My Drive/ThesisStoryGen/Data/ImgCapTraining/CocoTrain2017_1/'
TARGETDIR = r'./gdrive/My Drive/ThesisStoryGen/Data/coco_train2017_wget_1/' ## the dedicated folder for subset of images will be made here

In [4]:
## reload from picked file - output of listdir on google drive location = r'./gdrive/My Drive/ThesisStoryGen/Data/coco_train2017_wget_1/Rishabh_DO_NOT_TOUCH_PLS_1/'
if True:
  with open(WORKDIR_COCO_TRAIN2017+'gdrive_coco_train2017_118287_listdir.pkl', 'rb') as handle:
    gdrive_coco_train2017_118287_listdir = pickle.load(handle)

len(gdrive_coco_train2017_118287_listdir)

118287

In [5]:
gdrive_coco_train2017_118287_listdir[:3]

['000000407952.jpg', '000000031471.jpg', '000000208524.jpg']

In [6]:
## add the full path to the file names
gdrive_coco_train2017_118287_listdir = [COCO_TRAIN2017_IMGSDIR + f for f in gdrive_coco_train2017_118287_listdir]
gdrive_coco_train2017_118287_listdir[:3]

['./gdrive/My Drive/ThesisStoryGen/Data/coco_train2017_wget_1/Rishabh_DO_NOT_TOUCH_PLS_1/000000407952.jpg',
 './gdrive/My Drive/ThesisStoryGen/Data/coco_train2017_wget_1/Rishabh_DO_NOT_TOUCH_PLS_1/000000031471.jpg',
 './gdrive/My Drive/ThesisStoryGen/Data/coco_train2017_wget_1/Rishabh_DO_NOT_TOUCH_PLS_1/000000208524.jpg']

In [7]:
## slicing will be done on the gdrive_coco_train2017_118287_listdir array
## i.e. gdrive_coco_train2017_118287_listdir[ sidx : eidx ]

In [8]:
def copy_the_files(_sidx, _eidx, _files_arr, _copyTo_path, _print_freq = 50):
    start_tick = time.time()
    cnt_processed , cnt_copied , cnt_problem = 0, 0, 0
    problem_files = []
    for file_to_be_copied in _files_arr[_sidx : _eidx]:
      if cnt_processed % _print_freq == 0:
        print(f"At {datetime.datetime.now().strftime('%H:%M:%S')} :: Processed = {cnt_processed} , Copied = {cnt_copied} , Problem = {cnt_problem}")
      cnt_processed += 1
      try:
        shutil.copy(file_to_be_copied, _copyTo_path)
        cnt_copied += 1
      except Exception as e:
        cnt_problem += 1
        problem_files.append(file_to_be_copied)
        #print(f"Error: {e}")
    
    print(f"\n\nExpected count of copied files = {_eidx - _sidx}\nTime taken = { time.time() - start_tick } seconds")
    print(f"\nProcessed = {cnt_processed} , Copied = {cnt_copied} , Problem = {cnt_problem}\n")
    
    return problem_files

### copying   for indexes     start = 0 , end = 5000

In [ ]:
sidx, eidx = 0, 5000

In [ ]:
copyTo_path = TARGETDIR + 'coco_test2017_subset_' + str(sidx) + '_' + str(eidx) + '/'
copyTo_path

'./gdrive/My Drive/ThesisStoryGen/Data/coco_train2017_wget_1/coco_test2017_subset_0_5000/'

In [ ]:
if not os.path.isdir(copyTo_path):
    os.mkdir(copyTo_path)
    print(f"\n\nCreated folder on Google Drive for copyTo_path : {copyTo_path}\n\n")
    
    problem_files_arr = copy_the_files(sidx, eidx, gdrive_coco_train2017_118287_listdir, copyTo_path)
else:
    print(f"\n\nFATAL ERROR: copyTo_path already exists : {copyTo_path}\n\n")

if len(problem_files_arr) > 0:
  print(f"\n\nThere were problems.\n")
else:
  print(f"\n\nSuccess, no problems\n")



Created folder on Google Drive for copyTo_path : ./gdrive/My Drive/ThesisStoryGen/Data/coco_train2017_wget_1/coco_test2017_subset_0_5000/


At 16:12:55 :: Processed = 0 , Copied = 0 , Problem = 0
At 16:12:55 :: Processed = 50 , Copied = 50 , Problem = 0
At 16:12:56 :: Processed = 100 , Copied = 100 , Problem = 0
At 16:12:56 :: Processed = 150 , Copied = 150 , Problem = 0
At 16:12:57 :: Processed = 200 , Copied = 200 , Problem = 0
At 16:12:57 :: Processed = 250 , Copied = 250 , Problem = 0
At 16:12:58 :: Processed = 300 , Copied = 300 , Problem = 0
At 16:12:58 :: Processed = 350 , Copied = 350 , Problem = 0
At 16:12:59 :: Processed = 400 , Copied = 400 , Problem = 0
At 16:12:59 :: Processed = 450 , Copied = 450 , Problem = 0
At 16:13:00 :: Processed = 500 , Copied = 500 , Problem = 0
At 16:13:00 :: Processed = 550 , Copied = 550 , Problem = 0
At 16:13:01 :: Processed = 600 , Copied = 600 , Problem = 0
At 16:13:02 :: Processed = 650 , Copied = 650 , Problem = 0
At 16:13:02 :: Processed

In [ ]:
len(os.listdir('./gdrive/My Drive/ThesisStoryGen/Data/coco_train2017_wget_1/coco_test2017_subset_0_5000/'))

5000

### copying   for indexes     start = 5000 , end = 10000

In [ ]:
sidx, eidx = 5000, 10000

In [ ]:
copyTo_path = TARGETDIR + 'coco_test2017_subset_' + str(sidx) + '_' + str(eidx) + '/'
copyTo_path

'./gdrive/My Drive/ThesisStoryGen/Data/coco_train2017_wget_1/coco_test2017_subset_5000_10000/'

In [ ]:
if not os.path.isdir(copyTo_path):
    os.mkdir(copyTo_path)
    print(f"\n\nCreated folder on Google Drive for copyTo_path : {copyTo_path}\n\n")
    
    problem_files_arr = copy_the_files(sidx, eidx, gdrive_coco_train2017_118287_listdir, copyTo_path)
else:
    print(f"\n\nFATAL ERROR: copyTo_path already exists : {copyTo_path}\n\n")

if len(problem_files_arr) > 0:
  print(f"\n\nThere were problems.\n")
else:
  print(f"\n\nSuccess, no problems\n")



Created folder on Google Drive for copyTo_path : ./gdrive/My Drive/ThesisStoryGen/Data/coco_train2017_wget_1/coco_test2017_subset_5000_10000/


At 16:26:34 :: Processed = 0 , Copied = 0 , Problem = 0
At 16:26:48 :: Processed = 50 , Copied = 50 , Problem = 0
At 16:27:01 :: Processed = 100 , Copied = 100 , Problem = 0
At 16:27:15 :: Processed = 150 , Copied = 150 , Problem = 0
At 16:27:29 :: Processed = 200 , Copied = 200 , Problem = 0
At 16:27:45 :: Processed = 250 , Copied = 250 , Problem = 0
At 16:27:59 :: Processed = 300 , Copied = 300 , Problem = 0
At 16:28:14 :: Processed = 350 , Copied = 350 , Problem = 0
At 16:28:30 :: Processed = 400 , Copied = 400 , Problem = 0
At 16:28:45 :: Processed = 450 , Copied = 450 , Problem = 0
At 16:28:59 :: Processed = 500 , Copied = 500 , Problem = 0
At 16:29:12 :: Processed = 550 , Copied = 550 , Problem = 0
At 16:29:27 :: Processed = 600 , Copied = 600 , Problem = 0
At 16:29:41 :: Processed = 650 , Copied = 650 , Problem = 0
At 16:29:55 :: Proce

In [ ]:
len(os.listdir('./gdrive/My Drive/ThesisStoryGen/Data/coco_train2017_wget_1/coco_test2017_subset_5000_10000/'))

5000

### copying   for indexes     start = 10000 , end = 15000

In [ ]:
sidx, eidx = 10000, 15000

In [ ]:
copyTo_path = TARGETDIR + 'coco_test2017_subset_' + str(sidx) + '_' + str(eidx) + '/'
copyTo_path

'./gdrive/My Drive/ThesisStoryGen/Data/coco_train2017_wget_1/coco_test2017_subset_10000_15000/'

In [ ]:
if not os.path.isdir(copyTo_path):
    os.mkdir(copyTo_path)
    print(f"\n\nCreated folder on Google Drive for copyTo_path : {copyTo_path}\n\n")
    
    problem_files_arr = copy_the_files(sidx, eidx, gdrive_coco_train2017_118287_listdir, copyTo_path)
else:
    print(f"\n\nFATAL ERROR: copyTo_path already exists : {copyTo_path}\n\n")

if len(problem_files_arr) > 0:
  print(f"\n\nThere were problems.\n")
else:
  print(f"\n\nSuccess, no problems\n")



Created folder on Google Drive for copyTo_path : ./gdrive/My Drive/ThesisStoryGen/Data/coco_train2017_wget_1/coco_test2017_subset_10000_15000/


At 16:51:33 :: Processed = 0 , Copied = 0 , Problem = 0
At 16:51:47 :: Processed = 50 , Copied = 50 , Problem = 0
At 16:52:01 :: Processed = 100 , Copied = 100 , Problem = 0
At 16:52:14 :: Processed = 150 , Copied = 150 , Problem = 0
At 16:52:28 :: Processed = 200 , Copied = 200 , Problem = 0
At 16:52:42 :: Processed = 250 , Copied = 250 , Problem = 0
At 16:52:55 :: Processed = 300 , Copied = 300 , Problem = 0
At 16:53:08 :: Processed = 350 , Copied = 350 , Problem = 0
At 16:53:21 :: Processed = 400 , Copied = 400 , Problem = 0
At 16:53:34 :: Processed = 450 , Copied = 450 , Problem = 0
At 16:53:49 :: Processed = 500 , Copied = 500 , Problem = 0
At 16:54:03 :: Processed = 550 , Copied = 550 , Problem = 0
At 16:54:17 :: Processed = 600 , Copied = 600 , Problem = 0
At 16:54:35 :: Processed = 650 , Copied = 650 , Problem = 0
At 16:54:48 :: Proc

In [ ]:
len(os.listdir('./gdrive/My Drive/ThesisStoryGen/Data/coco_train2017_wget_1/coco_test2017_subset_10000_15000/'))

5000

### copying   for indexes     start = 15000 , end = 20000

In [ ]:
sidx, eidx = 15000, 20000

In [ ]:
copyTo_path = TARGETDIR + 'coco_test2017_subset_' + str(sidx) + '_' + str(eidx) + '/'
copyTo_path

'./gdrive/My Drive/ThesisStoryGen/Data/coco_train2017_wget_1/coco_test2017_subset_15000_20000/'

In [ ]:
if not os.path.isdir(copyTo_path):
    os.mkdir(copyTo_path)
    print(f"\n\nCreated folder on Google Drive for copyTo_path : {copyTo_path}\n\n")
    
    problem_files_arr = copy_the_files(sidx, eidx, gdrive_coco_train2017_118287_listdir, copyTo_path)
else:
    print(f"\n\nFATAL ERROR: copyTo_path already exists : {copyTo_path}\n\n")

if len(problem_files_arr) > 0:
  print(f"\n\nThere were problems.\n")
else:
  print(f"\n\nSuccess, no problems\n")



Created folder on Google Drive for copyTo_path : ./gdrive/My Drive/ThesisStoryGen/Data/coco_train2017_wget_1/coco_test2017_subset_15000_20000/


At 17:18:41 :: Processed = 0 , Copied = 0 , Problem = 0
At 17:18:56 :: Processed = 50 , Copied = 50 , Problem = 0
At 17:19:10 :: Processed = 100 , Copied = 100 , Problem = 0
At 17:19:24 :: Processed = 150 , Copied = 150 , Problem = 0
At 17:19:40 :: Processed = 200 , Copied = 200 , Problem = 0
At 17:19:54 :: Processed = 250 , Copied = 250 , Problem = 0
At 17:20:08 :: Processed = 300 , Copied = 300 , Problem = 0
At 17:20:22 :: Processed = 350 , Copied = 350 , Problem = 0
At 17:20:36 :: Processed = 400 , Copied = 400 , Problem = 0
At 17:20:49 :: Processed = 450 , Copied = 450 , Problem = 0
At 17:21:02 :: Processed = 500 , Copied = 500 , Problem = 0
At 17:21:17 :: Processed = 550 , Copied = 550 , Problem = 0
At 17:21:29 :: Processed = 600 , Copied = 600 , Problem = 0
At 17:21:42 :: Processed = 650 , Copied = 650 , Problem = 0
At 17:21:55 :: Proc

In [ ]:
len(os.listdir('./gdrive/My Drive/ThesisStoryGen/Data/coco_train2017_wget_1/coco_test2017_subset_15000_20000/'))

5000

### copying   for indexes     start = 20000 , end = 25000 - WIP

In [9]:
sidx, eidx = 20000, 25000

In [10]:
copyTo_path = TARGETDIR + 'coco_test2017_subset_' + str(sidx) + '_' + str(eidx) + '/'
copyTo_path

'./gdrive/My Drive/ThesisStoryGen/Data/coco_train2017_wget_1/coco_test2017_subset_20000_25000/'

In [ ]:
if not os.path.isdir(copyTo_path):
    os.mkdir(copyTo_path)
    print(f"\n\nCreated folder on Google Drive for copyTo_path : {copyTo_path}\n\n")
    
    problem_files_arr = copy_the_files(sidx, eidx, gdrive_coco_train2017_118287_listdir, copyTo_path)
else:
    print(f"\n\nFATAL ERROR: copyTo_path already exists : {copyTo_path}\n\n")

if len(problem_files_arr) > 0:
  print(f"\n\nThere were problems.\n")
else:
  print(f"\n\nSuccess, no problems\n")



Created folder on Google Drive for copyTo_path : ./gdrive/My Drive/ThesisStoryGen/Data/coco_train2017_wget_1/coco_test2017_subset_20000_25000/


At 12:53:55 :: Processed = 0 , Copied = 0 , Problem = 0


In [ ]:
len(os.listdir('./gdrive/My Drive/ThesisStoryGen/Data/coco_train2017_wget_1/coco_test2017_subset_20000_25000/'))

### copying   for indexes     start = 25000 , end = 30000 - PENDING

In [ ]:
sidx, eidx = 25000, 30000

In [ ]:
copyTo_path = TARGETDIR + 'coco_test2017_subset_' + str(sidx) + '_' + str(eidx) + '/'
copyTo_path

In [ ]:
if not os.path.isdir(copyTo_path):
    os.mkdir(copyTo_path)
    print(f"\n\nCreated folder on Google Drive for copyTo_path : {copyTo_path}\n\n")
    
    problem_files_arr = copy_the_files(sidx, eidx, gdrive_coco_train2017_118287_listdir, copyTo_path)
else:
    print(f"\n\nFATAL ERROR: copyTo_path already exists : {copyTo_path}\n\n")

if len(problem_files_arr) > 0:
  print(f"\n\nThere were problems.\n")
else:
  print(f"\n\nSuccess, no problems\n")

In [ ]:
print("Count of file in :: {copyTo_path} = {len(os.listdir(copyTo_path))} ")

### copying   for indexes     start = 30000 , end = 35000 - PENDING

In [ ]:
sidx, eidx = 30000, 35000

In [ ]:
copyTo_path = TARGETDIR + 'coco_test2017_subset_' + str(sidx) + '_' + str(eidx) + '/'
copyTo_path

In [ ]:
if not os.path.isdir(copyTo_path):
    os.mkdir(copyTo_path)
    print(f"\n\nCreated folder on Google Drive for copyTo_path : {copyTo_path}\n\n")
    
    problem_files_arr = copy_the_files(sidx, eidx, gdrive_coco_train2017_118287_listdir, copyTo_path)
else:
    print(f"\n\nFATAL ERROR: copyTo_path already exists : {copyTo_path}\n\n")

if len(problem_files_arr) > 0:
  print(f"\n\nThere were problems.\n")
else:
  print(f"\n\nSuccess, no problems\n")

In [ ]:
print("Count of file in :: {copyTo_path} = {len(os.listdir(copyTo_path))} ")

### copying   for indexes     start = 35000 , end = 40000 - PENDING

In [ ]:
sidx, eidx = 35000, 40000

In [ ]:
copyTo_path = TARGETDIR + 'coco_test2017_subset_' + str(sidx) + '_' + str(eidx) + '/'
copyTo_path

In [ ]:
if not os.path.isdir(copyTo_path):
    os.mkdir(copyTo_path)
    print(f"\n\nCreated folder on Google Drive for copyTo_path : {copyTo_path}\n\n")
    
    problem_files_arr = copy_the_files(sidx, eidx, gdrive_coco_train2017_118287_listdir, copyTo_path)
else:
    print(f"\n\nFATAL ERROR: copyTo_path already exists : {copyTo_path}\n\n")

if len(problem_files_arr) > 0:
  print(f"\n\nThere were problems.\n")
else:
  print(f"\n\nSuccess, no problems\n")

In [ ]:
print("Count of file in :: {copyTo_path} = {len(os.listdir(copyTo_path))} ")

### copying   for indexes     start = 40000 , end = 45000 - PENDING

In [ ]:
sidx, eidx = 40000, 45000

In [ ]:
copyTo_path = TARGETDIR + 'coco_test2017_subset_' + str(sidx) + '_' + str(eidx) + '/'
copyTo_path

In [ ]:
if not os.path.isdir(copyTo_path):
    os.mkdir(copyTo_path)
    print(f"\n\nCreated folder on Google Drive for copyTo_path : {copyTo_path}\n\n")
    
    problem_files_arr = copy_the_files(sidx, eidx, gdrive_coco_train2017_118287_listdir, copyTo_path)
else:
    print(f"\n\nFATAL ERROR: copyTo_path already exists : {copyTo_path}\n\n")

if len(problem_files_arr) > 0:
  print(f"\n\nThere were problems.\n")
else:
  print(f"\n\nSuccess, no problems\n")

In [ ]:
print("Count of file in :: {copyTo_path} = {len(os.listdir(copyTo_path))} ")

### copying   for indexes     start = 45000 , end = 50000 - PENDING

In [ ]:
sidx, eidx = 45000, 50000

In [ ]:
copyTo_path = TARGETDIR + 'coco_test2017_subset_' + str(sidx) + '_' + str(eidx) + '/'
copyTo_path

In [ ]:
if not os.path.isdir(copyTo_path):
    os.mkdir(copyTo_path)
    print(f"\n\nCreated folder on Google Drive for copyTo_path : {copyTo_path}\n\n")
    
    problem_files_arr = copy_the_files(sidx, eidx, gdrive_coco_train2017_118287_listdir, copyTo_path)
else:
    print(f"\n\nFATAL ERROR: copyTo_path already exists : {copyTo_path}\n\n")

if len(problem_files_arr) > 0:
  print(f"\n\nThere were problems.\n")
else:
  print(f"\n\nSuccess, no problems\n")

In [ ]:
print("Count of file in :: {copyTo_path} = {len(os.listdir(copyTo_path))} ")

### copying   for indexes     start = 50000 , end = 55000 - PENDING

In [ ]:
sidx, eidx = 50000, 55000

In [ ]:
copyTo_path = TARGETDIR + 'coco_test2017_subset_' + str(sidx) + '_' + str(eidx) + '/'
copyTo_path

In [ ]:
if not os.path.isdir(copyTo_path):
    os.mkdir(copyTo_path)
    print(f"\n\nCreated folder on Google Drive for copyTo_path : {copyTo_path}\n\n")
    
    problem_files_arr = copy_the_files(sidx, eidx, gdrive_coco_train2017_118287_listdir, copyTo_path)
else:
    print(f"\n\nFATAL ERROR: copyTo_path already exists : {copyTo_path}\n\n")

if len(problem_files_arr) > 0:
  print(f"\n\nThere were problems.\n")
else:
  print(f"\n\nSuccess, no problems\n")

In [ ]:
print("Count of file in :: {copyTo_path} = {len(os.listdir(copyTo_path))} ")

### copying   for indexes     start = 55000 , end = 60000 - PENDING

In [ ]:
sidx, eidx = 55000, 60000

In [ ]:
copyTo_path = TARGETDIR + 'coco_test2017_subset_' + str(sidx) + '_' + str(eidx) + '/'
copyTo_path

In [ ]:
if not os.path.isdir(copyTo_path):
    os.mkdir(copyTo_path)
    print(f"\n\nCreated folder on Google Drive for copyTo_path : {copyTo_path}\n\n")
    
    problem_files_arr = copy_the_files(sidx, eidx, gdrive_coco_train2017_118287_listdir, copyTo_path)
else:
    print(f"\n\nFATAL ERROR: copyTo_path already exists : {copyTo_path}\n\n")

if len(problem_files_arr) > 0:
  print(f"\n\nThere were problems.\n")
else:
  print(f"\n\nSuccess, no problems\n")

In [ ]:
print("Count of file in :: {copyTo_path} = {len(os.listdir(copyTo_path))} ")

### copying   for indexes     start = 60000 , end = 65000 - PENDING

In [ ]:
sidx, eidx = 60000, 65000

In [ ]:
copyTo_path = TARGETDIR + 'coco_test2017_subset_' + str(sidx) + '_' + str(eidx) + '/'
copyTo_path

In [ ]:
if not os.path.isdir(copyTo_path):
    os.mkdir(copyTo_path)
    print(f"\n\nCreated folder on Google Drive for copyTo_path : {copyTo_path}\n\n")
    
    problem_files_arr = copy_the_files(sidx, eidx, gdrive_coco_train2017_118287_listdir, copyTo_path)
else:
    print(f"\n\nFATAL ERROR: copyTo_path already exists : {copyTo_path}\n\n")

if len(problem_files_arr) > 0:
  print(f"\n\nThere were problems.\n")
else:
  print(f"\n\nSuccess, no problems\n")

In [ ]:
print("Count of file in :: {copyTo_path} = {len(os.listdir(copyTo_path))} ")

### copying   for indexes     start = 65000 , end = 70000 - PENDING

In [ ]:
sidx, eidx = 65000, 70000

In [ ]:
copyTo_path = TARGETDIR + 'coco_test2017_subset_' + str(sidx) + '_' + str(eidx) + '/'
copyTo_path

In [ ]:
if not os.path.isdir(copyTo_path):
    os.mkdir(copyTo_path)
    print(f"\n\nCreated folder on Google Drive for copyTo_path : {copyTo_path}\n\n")
    
    problem_files_arr = copy_the_files(sidx, eidx, gdrive_coco_train2017_118287_listdir, copyTo_path)
else:
    print(f"\n\nFATAL ERROR: copyTo_path already exists : {copyTo_path}\n\n")

if len(problem_files_arr) > 0:
  print(f"\n\nThere were problems.\n")
else:
  print(f"\n\nSuccess, no problems\n")

In [ ]:
print("Count of file in :: {copyTo_path} = {len(os.listdir(copyTo_path))} ")

### copying   for indexes     start = 70000 , end = 75000 - PENDING

In [ ]:
sidx, eidx = 70000, 75000

In [ ]:
copyTo_path = TARGETDIR + 'coco_test2017_subset_' + str(sidx) + '_' + str(eidx) + '/'
copyTo_path

In [ ]:
if not os.path.isdir(copyTo_path):
    os.mkdir(copyTo_path)
    print(f"\n\nCreated folder on Google Drive for copyTo_path : {copyTo_path}\n\n")
    
    problem_files_arr = copy_the_files(sidx, eidx, gdrive_coco_train2017_118287_listdir, copyTo_path)
else:
    print(f"\n\nFATAL ERROR: copyTo_path already exists : {copyTo_path}\n\n")

if len(problem_files_arr) > 0:
  print(f"\n\nThere were problems.\n")
else:
  print(f"\n\nSuccess, no problems\n")

In [ ]:
print("Count of file in :: {copyTo_path} = {len(os.listdir(copyTo_path))} ")

## NEXT SECTION - PENDING